In [4]:
# DFS
import queue
import time

def is_valid_move(x, y, visited, graph):
    if x < 0 or y < 0 or x >= len(graph) or y >= len(graph[0]):
        return False
    if visited[x][y] or graph[x][y] == "#":
        return False
    return True

def dfs(graph, start):
    # inisialisasi variabel
    visited = [[False]*len(graph[0]) for _ in range(len(graph))]
    stack = [(start[0], start[1], [start])]
    paths = []

    # mulai pencarian
    while stack:
        x, y, path = stack.pop()
        if graph[x][y] == "$":
            paths.append(path)
            continue
        if visited[x][y]:
            continue

        visited[x][y] = True
        for i, j in [(x-1, y), (x+1, y), (x, y-1), (x, y+1)]:
            if is_valid_move(i, j, visited, graph):
                stack.append((i, j, path+[(i, j)]))

    return paths

if __name__ == "__main__":
    with open("input.txt") as f:
        graph = [line.strip() for line in f.readlines()]
    start = None
    for i in range(len(graph)):
        for j in range(len(graph[0])):
            if graph[i][j] == "@":
                start = (i, j)
                break
        if start:
            break

    paths = dfs(graph, start)
    for path in paths:
      start_time = time.time()
    print()
    print(path)
    print()
    end_time = time.time()
    print()
    print('Steps to goal: {0}'.format(len(path)))
    print()
    print("time executing : " , end_time - start_time)




[(39, 40), (39, 39), (39, 38), (39, 37), (39, 36), (39, 35), (39, 34), (39, 33), (38, 33), (37, 33), (37, 32), (37, 31), (38, 31), (39, 31), (39, 30), (39, 29), (38, 29), (37, 29), (36, 29), (35, 29), (34, 29), (33, 29), (32, 29), (31, 29), (30, 29), (29, 29), (29, 28), (29, 27), (28, 27), (27, 27), (26, 27), (25, 27), (25, 26), (25, 25), (25, 24), (25, 23), (25, 22), (25, 21), (26, 21), (27, 21), (28, 21), (29, 21), (29, 22), (29, 23), (30, 23), (31, 23), (31, 24), (31, 25), (32, 25), (33, 25), (34, 25), (35, 25), (36, 25), (37, 25), (38, 25), (39, 25), (39, 24), (39, 23), (39, 22), (39, 21), (38, 21), (37, 21), (36, 21), (35, 21), (35, 22), (35, 23), (34, 23), (33, 23), (33, 22), (33, 21), (32, 21), (31, 21), (31, 20), (31, 19), (32, 19), (33, 19), (34, 19), (35, 19), (35, 18), (35, 17), (34, 17), (33, 17), (32, 17), (31, 17), (30, 17), (29, 17), (29, 16), (29, 15), (30, 15), (31, 15), (32, 15), (33, 15), (33, 14), (33, 13), (34, 13), (35, 13), (35, 14), (35, 15), (36, 15), (37, 15)

In [5]:
# Greedy Best First Search Algorithm
import time
# This class represents a node
class Node:
    # Initialize the class
    def __init__(self, position, parent):
        self.position = position
        self.parent = parent
        self.g = 0  # Distance to start node
        self.h = 0  # Distance to goal node
        self.f = 0  # Total cost

    # Compare nodes
    def __eq__(self, other):
        return self.position == other.position

    # Sort nodes
    def __lt__(self, other):
        return self.f < other.f

    # Print node
    def __repr__(self):
        return f"({self.position}, {self.f})"

# Draw a grid
def draw_grid(map, width, height, spacing=2, **kwargs):
    for y in range(height):
        for x in range(width):
            print(f"%-{spacing}s" % draw_tile(map, (x, y), kwargs), end="")
        print()

# Draw a tile
def draw_tile(map, position, kwargs):
    # Get the map value
    value = map.get(position)

    # Check if we should print the path
    if "path" in kwargs and position in kwargs["path"]:
        value = "+"

    # Check if we should print start point
    if "start" in kwargs and position == kwargs["start"]:
        value = "@"

    # Check if we should print the goal point
    if "goal" in kwargs and position == kwargs["goal"]:
        value = "$"

    # Return a tile value
    return value

# Best-first search
def best_first_search(map, start, end):
    # Create lists for open nodes and closed nodes
    open = []
    closed = []

    # Create a start node and a goal node
    start_node = Node(start, None)
    goal_node = Node(end, None)

    # Add the start node
    open.append(start_node)

    # Loop until the open list is empty
    while len(open) > 0:
        # Sort the open list to get the node with the lowest cost first
        open.sort()

        # Get the node with the lowest cost
        current_node = open.pop(0)

        # Add the current node to the closed list
        closed.append(current_node)

        # Check if we have reached the goal, return the path
        if current_node == goal_node:
            path = []
            while current_node != start_node:
                path.append(current_node.position)
                current_node = current_node.parent

            # Reverse the path and return it
            return path[::-1]

        # Unzip the current node position
        (x, y) = current_node.position

        # Get neighbors
        neighbors = [(x-1, y), (x+1, y), (x, y-1), (x, y+1)]

        # Loop neighbors
        for next in neighbors:
            # Get value from map
            map_value = map.get(next)

            # Check if the node is a wall
            if map_value == "#":
                continue

            # Create a neighbor node
            neighbor = Node(next, current_node)

            # Check if the neighbor is in the closed list
            if neighbor in closed:
                continue
            # Generate heuristics (Manhattan distance)
            neighbor.g = abs(neighbor.position[0] - start_node.position[0]) + abs(neighbor.position[1] - start_node.position[1])
            neighbor.h = abs(neighbor.position[0] - goal_node.position[0]) + abs(neighbor.position[1] - goal_node.position[1])
            neighbor.f = neighbor.h 
            # Check if neighbor is in open list and if it has a lower f value
            if(add_to_open(open, neighbor) == True):
                # Everything is green, add neighbor to open list
                open.append(neighbor)
    # Return None, no path is found
    return None
# Check if a neighbor should be added to open list
def add_to_open(open, neighbor):
    for node in open:
        if (neighbor == node and neighbor.f >= node.f):
            return False
    return True
# The main entry point for this module
def main():
    # Get a map (grid)
    map = {}
    chars = ['c']
    start = None
    end = None
    width = 0
    height = 0
    # Open a file
    fp = open('input.txt', 'r')
    # fp = open('data\\input.in', 'r')
    
    # Loop until there is no more lines
    while len(chars) > 0:
        # Get chars in a line
        chars = [str(i) for i in fp.readline().strip()]
        # Calculate the width
        width = len(chars) if width == 0 else width
        # Add chars to map
        for x in range(len(chars)):
            map[(x, height)] = chars[x]
            if(chars[x] == '@'):
                start = (x, height)
            elif(chars[x] == '$'):
                end = (x, height)
        
        # Increase the height of the map
        if(len(chars) > 0):
            height += 1
    # Close the file pointer
    fp.close()
    start_time = time.time()
    # Find the closest path from start(@) to end($)
    path = best_first_search(map, start, end)
    print()
    print(path)
    print()
    draw_grid(map, width, height, spacing=1, path=path, start=start, goal=end)
    end_time = time.time()
    print()
    print('Steps to goal: {0}'.format(len(path)))
    print()
    print("time executing : " , end_time - start_time)
# Tell python to run main method
if __name__ == "__main__": main()


[(39, 39), (38, 39), (37, 39), (36, 39), (35, 39), (34, 39), (33, 39), (33, 38), (33, 37), (32, 37), (31, 37), (31, 38), (31, 39), (30, 39), (29, 39), (29, 38), (29, 37), (29, 36), (29, 35), (29, 34), (29, 33), (29, 32), (29, 31), (29, 30), (29, 29), (28, 29), (27, 29), (27, 28), (27, 27), (27, 26), (27, 25), (26, 25), (25, 25), (24, 25), (23, 25), (22, 25), (21, 25), (21, 26), (21, 27), (21, 28), (21, 29), (22, 29), (23, 29), (23, 30), (23, 31), (24, 31), (25, 31), (25, 32), (25, 33), (25, 34), (25, 35), (25, 36), (25, 37), (25, 38), (25, 39), (24, 39), (23, 39), (22, 39), (21, 39), (21, 38), (21, 37), (21, 36), (21, 35), (22, 35), (23, 35), (23, 34), (23, 33), (22, 33), (21, 33), (21, 32), (21, 31), (20, 31), (19, 31), (19, 32), (19, 33), (19, 34), (19, 35), (18, 35), (17, 35), (17, 34), (17, 33), (17, 32), (17, 31), (17, 30), (17, 29), (16, 29), (15, 29), (15, 30), (15, 31), (15, 32), (15, 33), (14, 33), (13, 33), (13, 34), (13, 35), (14, 35), (15, 35), (15, 36), (15, 37), (16, 37)